In [2]:
%pip install flask_cors

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np

from flask import Flask, request, jsonify
from flask_cors import CORS

import json
import logging

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

app = Flask(__name__)
CORS(app)
logging.basicConfig(level=logging.DEBUG)

valData = pd.read_csv('./nValData.csv').drop(columns=['Unnamed: 0'])

def weightScore(chainScore, weight):
    chainScore['contribution_score'] = chainScore['contribution_score'] * weight[0] / sum(weight)
    chainScore['stability_score'] = chainScore['stability_score'] * weight[1] / sum(weight)
    chainScore['popularity_score'] = chainScore['popularity_score'] * weight[2] / sum(weight)
    chainScore['commission_score'] = chainScore['commission_score'] * weight[3] / sum(weight)
    chainScore['period_score'] = chainScore['period_score'] * weight[4] / sum(weight)
    chainScore['total_score'] = chainScore['contribution_score'] + chainScore['stability_score'] + chainScore['popularity_score'] + chainScore['commission_score'] + chainScore['period_score']
    return chainScore

def calculate_inscore(row, score_condition, missblock, jailed, token_outlier, participation):
    score = 0
    additional_score = 0
    additional_conditions = [0] * 5
    num=0

    # 조건 1: 점수 지표가 ?점보다 높은가?
    if row['total_score'] >= score_condition:
        score += 20
        num += 1
        if score_condition != 100:
            additional_conditions[0] = (row['total_score'] - score_condition) / (100 - score_condition)
        else:
            additional_conditions[0] = 1

    # 조건 2: missblock 개수가 ?개보다 작은가?
    if row['missblock'] <= missblock :
        score += 20
        num += 1
        if missblock != 0:
            additional_conditions[1] = (missblock - row['missblock']) / missblock
        else:
            additional_conditions[1] = 1
        
    # 조건 3: 기간 내 jailed 비율이 ?보다 작은가?
    if row['jailed_ratio'] <= jailed :
        score += 20
        num += 1
        if jailed != 0:
            additional_conditions[2] = (jailed - row['jailed_ratio']) / jailed
        else:
            additional_conditions[2] = 1

    # 조건 4: 토큰 변화량 이상치가 ?보다 작은가?
    if row['n_outlier'] <= token_outlier :
        score += 20
        num += 1
        if token_outlier != 0:
            additional_conditions[3] = (token_outlier - row['n_outlier']) / token_outlier
        else:
            additional_conditions[3] = 1

    # 조건 5: 활동 참여율이 ?이상인가?
    if row['p_participation'] >= participation:
        score += 20
        num += 1
        if participation != 1:
            additional_conditions[4] = (row['p_participation'] - participation) / (1 - participation)
        else:
            additional_conditions[4] = 1

    # 추가 점수 계산
    if num == 5:
        total_score = 100
    else:
        additional_score = sum(15 / 5 * condition for condition in additional_conditions)
        total_score = score + additional_score
        
    return total_score

def calculate_grade(score):
    if score < 20:
        return '비추천'
    elif score < 44:
        return '주의'
    elif score < 69:
        return '양호'
    else:
        return '추천'

def calculate_final(row, percentage):
    if row['proportion_av'] == 1:
        return row['in_score']
    else:
        return percentage[0] * row['in_score'] + percentage[1] * row['out_score']


@app.route('/')
def home():
    return jsonify(message="Proof of Validator")


@app.route('/getScore', methods=['POST'])
def getScore():
    try:
        data = request.get_json()

        chain = data['chain']
        weight = data['weight']

        chainData = valData[valData['chain']==chain]
        chainScore = chainData[['voter', 'contribution_score', 'stability_score', 'popularity_score', 'commission_score', 'period_score']]
        chainScore = weightScore(chainScore, weight).sort_values(by='total_score', ascending=False)

        json_str = chainScore.to_json(orient='records')

        return json_str, 200

    except Exception as e:
        print(f"An error occurred: {e}")
        return jsonify({"error": "Internal Server Error"}), 500


@app.route('/getValidator', methods=['POST'])
def getValidator():
    try:
        data = request.get_json()

        voter = data['voter']
        weight = data['weight']

        
        validator = valData[valData['voter']==voter]
        validator = weightScore(validator, weight).sort_values(by='total_score', ascending=False)

        asset_total = validator['asset_value'].sum()
        validator['asset_ratio'] = validator['asset_value'] / asset_total
        validator['weight_score'] = validator['total_score'] * validator['asset_ratio']

        validator = validator[['chain', 'contribution_score', 'stability_score', 'popularity_score', 'commission_score', 'period_score', 'total_score', 
                                'asset_value', 'asset_ratio', 'weight_score', 'rank', 'missblock', 'p_participation', 'p_passed', 'p_matchproposal', 'jailed_ratio']]

        json_str = validator.to_json(orient='records')

        return jsonify(json_str), 200

    except Exception as e:
        print(f"An error occurred: {e}")
        return jsonify({"error": "Internal Server Error"}), 500


@app.route('/similarity', methods=['POST'])
def similarity():
    try:
        data = request.get_json()

        chain = data['chain']
        weight = data['weight']
        score_condition = data['score_condition']
        missblock = data['missblock']
        jailed = data['jailed']
        token_outlier = data['token_outlier']
        participation = data['participation']
        only_in = data['only_in']
        percentage = data['percentage']


        valData = pd.read_csv('./nValData.csv').drop(columns=['Unnamed: 0'])

        # 가중치 점수 계산
        valData = weightScore(valData, weight)
        avg_score = (valData['proportion_av']*valData["total_score"]).groupby(valData['voter']).sum().reset_index()
        avg_score.rename(columns = {0 : 'avg_score'}, inplace = True)
        valData = pd.merge(valData,avg_score, on='voter', how='left')

        # in_score 계산
        valData['in_score'] = valData.apply(lambda row: calculate_inscore(row, score_condition, missblock, jailed, token_outlier, participation), axis=1)
        valData['grade'] = valData['in_score'].apply(calculate_grade)

        # out_score 계산
        selected_chain_data = valData[valData['chain'] == chain]
        other_chain_data = valData[valData['chain'] != chain]

        other_chain_scores = other_chain_data.groupby('voter')[['in_score', 'asset_value']].apply(
        lambda x: (x['in_score'] * (x['asset_value'] / (x['asset_value'].sum()))).sum())

        selected_chain_data['out_score'] = selected_chain_data['voter'].map(other_chain_scores.fillna(0))
        selected_chain_data['out_score'] = selected_chain_data['out_score'].fillna(0)

        # final_grade 계산
        if only_in:
            selected_chain_data['final_score'] = selected_chain_data.apply(lambda row: calculate_final(row, percentage), axis=1)
        else:
            selected_chain_data['final_score'] = percentage[0] * selected_chain_data['in_score'] + percentage[1] * selected_chain_data['out_score']

        selected_chain_data['final_grade'] = selected_chain_data['final_score'].apply(calculate_grade)

        # 코사인 유사도
        scv = selected_chain_data.copy()

        score_columns = [
            'contribution_score', 'stability_score', 'popularity_score', 
            'commission_score', 'period_score', 'final_score', 
            'missblock', 'jailed_ratio', 'p_participation', 'n_outlier'
        ]

        duplicates = scv['voter'][scv['voter'].duplicated()]
        if not duplicates.empty:
            print("Duplicate voters found:", duplicates)
            scv = scv.drop_duplicates(subset=['voter'])
        else:
            print("No duplicates found.")

        scores = scv[score_columns]

        scaler = StandardScaler()
        normalized_scores = scaler.fit_transform(scores)

        similarity_matrix = cosine_similarity(normalized_scores)
        similarity_df = pd.DataFrame(similarity_matrix, index=scv['voter'], columns=scv['voter'])


        scd = selected_chain_data[['voter', 'final_score', 'final_grade']].to_json(orient='records')
        sdf = similarity_df.to_json(orient='index')
        
        combined_json = {
            "data": json.loads(scd),
            "similarity": json.loads(sdf)
        }
        
        return jsonify(combined_json), 200

    except Exception as e:
        print(f"An error occurred: {e}")
        return jsonify({"error": "Internal Server Error"}), 500


@app.route('/getSimilarityInfo', methods=['POST'])
def getSimilarityInfo():
    try:
        data = request.get_json()

        chain = data['chain']
        voter = data['voter']
        weight = data['weight']
        score_condition = data['score_condition']
        missblock = data['missblock']
        jailed = data['jailed']
        token_outlier = data['token_outlier']
        participation = data['participation']
        only_in = data['only_in']
        percentage = data['percentage']

        valData = pd.read_csv('./nValData.csv').drop(columns=['Unnamed: 0'])

        # 가중치 점수 계산
        valData = weightScore(valData, weight)
        avg_score = (valData['proportion_av']*valData["total_score"]).groupby(valData['voter']).sum().reset_index()
        avg_score.rename(columns = {0 : 'avg_score'}, inplace = True)
        valData = pd.merge(valData,avg_score, on='voter', how='left')

        # in_score 계산
        valData['in_score'] = valData.apply(lambda row: calculate_inscore(row, score_condition, missblock, jailed, token_outlier, participation), axis=1)
        valData['grade'] = valData['in_score'].apply(calculate_grade)

        # out_score 계산
        selected_chain_data = valData[valData['chain'] == chain]
        other_chain_data = valData[valData['chain'] != chain]

        other_chain_scores = other_chain_data.groupby('voter')[['in_score', 'asset_value']].apply(
        lambda x: (x['in_score'] * (x['asset_value'] / (x['asset_value'].sum()))).sum())

        selected_chain_data['out_score'] = selected_chain_data['voter'].map(other_chain_scores.fillna(0))
        selected_chain_data['out_score'] = selected_chain_data['out_score'].fillna(0)

        # final_grade 계산
        if only_in:
            selected_chain_data['final_score'] = selected_chain_data.apply(lambda row: calculate_final(row, percentage), axis=1)
        else:
            selected_chain_data['final_score'] = percentage[0] * selected_chain_data['in_score'] + percentage[1] * selected_chain_data['out_score']

        selected_chain_data['final_grade'] = selected_chain_data['final_score'].apply(calculate_grade)

        # 코사인 유사도
        scv = selected_chain_data.copy()

        score_columns = [
            'contribution_score', 'stability_score', 'popularity_score', 
            'commission_score', 'period_score', 'final_score', 
            'missblock', 'jailed_ratio', 'p_participation', 'n_outlier'
        ]

        duplicates = scv['voter'][scv['voter'].duplicated()]
        if not duplicates.empty:
            print("Duplicate voters found:", duplicates)
            scv = scv.drop_duplicates(subset=['voter'])
        else:
            print("No duplicates found.")


        scores = scv[score_columns]

        scaler = StandardScaler()
        normalized_scores = scaler.fit_transform(scores)

        similarity_matrix = cosine_similarity(normalized_scores)
        similarity_df = pd.DataFrame(similarity_matrix, index=scv['voter'], columns=scv['voter'])


        simVal = pd.DataFrame(similarity_df[voter]).reset_index(drop=False).sort_values(by=voter, ascending=False).reset_index(drop=True)[:6]
        simVal = simVal[['voter']].merge(scv, how='inner', on='voter')

        columns = [
            'voter', 'final_score', 'contribution_score', 'stability_score', 
            'popularity_score', 'commission_score', 'period_score', 
            'missblock', 'jailed_ratio', 'p_participation', 'n_outlier'
        ]

        valInfo = simVal[columns].sort_values(by='final_score', ascending=False).reset_index(drop=True)

        voter_counts = valData['voter'].value_counts()
        valInfo['chain_num'] = valInfo['voter'].apply(lambda x: voter_counts.get(x, 0))

        vinfo = valInfo.to_json(orient='records')

        return vinfo, 200

    except Exception as e:
        print(f"An error occurred: {e}")
        return jsonify({"error": "Internal Server Error"}), 500


@app.route('/getRaw', methods=['POST'])
def getRaw():
    try:
        data = request.get_json()

        chain = data['chain']
        weight = data['weight']
        score_condition = data['score_condition']
        missblock = data['missblock']
        jailed = data['jailed']
        token_outlier = data['token_outlier']
        participation = data['participation']
        only_in = data['only_in']
        percentage = data['percentage']

        valData = pd.read_csv('./nValData.csv').drop(columns=['Unnamed: 0'])

        # 가중치 점수 계산
        valData = weightScore(valData, weight)
        avg_score = (valData['proportion_av']*valData["total_score"]).groupby(valData['voter']).sum().reset_index()
        avg_score.rename(columns = {0 : 'avg_score'}, inplace = True)
        valData = pd.merge(valData,avg_score, on='voter', how='left')

        # in_score 계산
        valData['in_score'] = valData.apply(lambda row: calculate_inscore(row, score_condition, missblock, jailed, token_outlier, participation), axis=1)
        valData['grade'] = valData['in_score'].apply(calculate_grade)

        # out_score 계산
        selected_chain_data = valData[valData['chain'] == chain]
        other_chain_data = valData[valData['chain'] != chain]

        other_chain_scores = other_chain_data.groupby('voter')[['in_score', 'asset_value']].apply(
        lambda x: (x['in_score'] * (x['asset_value'] / (x['asset_value'].sum()))).sum())

        selected_chain_data['out_score'] = selected_chain_data['voter'].map(other_chain_scores.fillna(0))
        selected_chain_data['out_score'] = selected_chain_data['out_score'].fillna(0)

        # final_grade 계산
        if only_in:
            selected_chain_data['final_score'] = selected_chain_data.apply(lambda row: calculate_final(row, percentage), axis=1)
        else:
            selected_chain_data['final_score'] = percentage[0] * selected_chain_data['in_score'] + percentage[1] * selected_chain_data['out_score']

        selected_chain_data['final_grade'] = selected_chain_data['final_score'].apply(calculate_grade)

        # raw data 추출
        scv = selected_chain_data.copy()

        raw_columns = [
            'voter', 'final_score', 'final_grade', 'total_score', 
            'in_score', 'out_score', 'rank', 'token', 'voting_power', 
            'asset_value', 'commission', 'delegator'
        ]
        scv = scv[raw_columns].sort_values(by='rank', ascending=False).reset_index(drop=True)
        scv = scv.to_json(orient='records')

        return scv, 200

    except Exception as e:
        print(f"An error occurred: {e}")
        return jsonify({"error": "Internal Server Error"}), 500

if __name__ == '__main__':
    app.run(port=5002)